In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml
import torch

boston = fetch_openml(name='boston', version=1)

X = boston.data.to_numpy().astype(float)
Y = boston.target.to_numpy().astype(float)

np.random.seed(123)
               torch.random.manual_seed(123)
